%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets

In [2]:
import sys

# setting path
sys.path.append('../client')
import serving_client
import game_Client 

In [3]:
#Here I am instanciating the classes from game_client and serving_client

pinger = game_Client.gameClient(ip="127.0.0.1",port=6758)
serving = serving_client.AppClient(ip="127.0.0.1",port=6758)

In [4]:
game = pinger.pingGame('2021020329')
print(game)

Empty DataFrame
Columns: [eventType, period, periodTime, periodType, gameID, teamOfShooter, homeOrAway, xCoord, yCoord, shooter, goalie, shotType, emptyNet, strength, season, rinkSide, gameType, totalPlayTime, lastEventType, lastEventPeriod, lastEventPeriodTime, lastEventXCoord, lastEventYCoord, is_goal, EmptyNet, distance, angle, gameSeconds, lastEventGameSeconds, timeFromLastEvent, distanceFromLastEvent, rebound, lastEventAngle, changeInAngleShot, speed, timeSincePowerPlayStarted, numFriendlyNonGoalieSkaters, numOpposingNonGoalieSkaters, strength2]
Index: []

[0 rows x 39 columns]


In [5]:
serving.predict(game,'2021020329')

[0.1373259574174881,
 0.11719298362731934,
 0.054750945419073105,
 0.021762799471616745,
 0.020969975739717484,
 0.111834816634655,
 0.04057128727436066,
 0.10517524182796478,
 0.03199763223528862,
 0.03911442309617996,
 0.056099627166986465,
 0.021959971636533737,
 0.08207261562347412,
 0.12972502410411835,
 0.03868458792567253,
 0.1373259574174881,
 0.16570928692817688,
 0.08837094902992249,
 0.09348900616168976,
 0.08067451417446136,
 0.026046667248010635,
 0.05270504206418991,
 0.12416189163923264,
 0.12735049426555634,
 0.03463427722454071,
 0.016764365136623383,
 0.04373400658369064,
 0.13980941474437714,
 0.003696165047585964,
 0.013908498920500278,
 0.010872800834476948,
 0.1870231181383133,
 0.08584775030612946,
 0.05907023698091507,
 0.04420512542128563,
 0.01959666609764099,
 0.08085787296295166,
 0.050502873957157135,
 0.06533315032720566,
 0.10178351402282715,
 0.11751626431941986,
 0.0372968427836895,
 0.22253085672855377,
 0.23231938481330872,
 0.05831390246748924,
 0.17

In [5]:
#Input text for the model
text1 = widgets.Text(
       value='workspaceTest',
       description='workspace',)
text2 = widgets.Text(
       value='modelTest',
       description='model',)
text3 = widgets.Text(
       value='versionTest',
       description='version',)

#Button1 will load the model of choice
button1 = widgets.Button(description='Load Model')

#Input text for gameID
text4 = widgets.Text(
       value='2021020329',
       description='gameID',) 

#Button2 will ping the game
button2 = widgets.Button(description='Ping Game')
out = widgets.Output()

#Here will be stored the info of games and gameID

memoryGame=0
memoryGameID=0

def on_button1_clicked(_):
    #This function activates when button 1 is pushed
      with out:
          clear_output()  
          #download_registry_model(self, workspace: str, model: str, version: str)
          print("Model '",text1.value, "' Version '",text2.value,"' Workspace '",text3.value," has been uploaded")
            
def on_button2_clicked(_):
      # This function activates when button 2 is pushed 
      with out:
          clear_output()
            
          #Ping game for a given gameID
          game = pinger.pingGame(str(text4.value))
          
          if memoryGameID==0:
              #if no memory yet, set memoryGameID
              memoryGameID=str(text4.value)
          if memoryGame==0:
              #if no memory yet, set memoryGame
              memoryGame=game
          else:
            if memoryGameID!=str(text4.value):
                #if the memoryGameID does not correspond to the input gameID, it means that
                #we are dealing with a new game. 
                memoryGame=game
                memoryGameID=str(text4.value)
            else:
                #This is if the new events belong to the same game
                memoryGame.append(game,ignore_index=True)
            

            
          #Get current period and time left for that period  
          period = memoryGame.iloc[-1]["period"]
          time = str(memoryGame.iloc[-1]["periodTime"]).split(":")
          minuteLeft = 19-int(time[0])
          secondLeft=60-int(time[1])
          timeLeft= str(minuteLeft)+":"+str(secondLeft)
        
          #Get teams and homeOrAway
          teams=list(memoryGame["teamOfShooter"])
          homeOrAway=list(memoryGame["homeOrAway"])
          teams = list(dict.fromkeys(teams))
          homeOrAway= list(dict.fromkeys(homeOrAway))
           
          #Separate dataframes for each team, and get their corresponding XG
          XGhome=serving.predict(game,teams[0])
          XGaway=serving.predict(game,teams[1])
            
          #sum of XG
          sumt1 = sum(XGhome)
          sumt2= sum(XGaway)
        
          #Output to print 
          print("period:",period)
          print("time left:",timeLeft)
          print(homeOrAway[0],"team and expected score:",teams[0],sumt1)
          print(homeOrAway[1],"team and expected score:",teams[1],sumt2)


        

#Associate buttons to functions
button1.on_click(on_button1_clicked)
button2.on_click(on_button2_clicked)
#Widgets that will be displayed
widgets.VBox([text1,text2,text3,button1,text4,button2,out])

    def ping_game(game_id, idx, other) -> new_events, new_idx, new_other
    
    def predict(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        Formats the inputs into an appropriate payload for a POST request, and queries the
        prediction service. Retrieves the response from the server, and processes it back into a
        dataframe that corresponds index-wise to the input dataframe.
        
        Args:
            X (Dataframe): Input dataframe to submit to the prediction service.
        """

        raise NotImplementedError("TODO: implement this function")

    def logs(self) -> dict:
        """Get server logs"""

        raise NotImplementedError("TODO: implement this function")

    def download_registry_model(self, workspace: str, model: str, version: str) -> dict:
        """
        Triggers a "model swap" in the service; the workspace, model, and model version are
        specified and the service looks for this model in the model registry and tries to
        download it. 

        See more here:

            https://www.comet.ml/docs/python-sdk/API/#apidownload_registry_model
        
        Args:
            workspace (str): The Comet ML workspace
            model (str): The model in the Comet ML registry to download
            version (str): The model version to download
        """

        raise NotImplementedError("TODO: implement this function")